# Fresh Idea
## separate one/zero activity of domains
- replace zeros by minus one
- calculate the class activity for 3 hours bins for each domain
- calculate the user activity for gaussian around center of 3 hour bins
- calculate the likelihood of the person being a 1/-1 in that time 
- add user general metrics including domain cls activity and usage patterns

### questions
- how to take into account times when the person used a website when others didnt?
- how to give likelihood when the person didn't show any nearby activity?
- what if he used a similar website at same time but more nich? 
- how to average the bins weighted by the significance of that bin?
- how to give weight to the magnitude of number of users entering? probability of 1 with confidence
- what about sparse websites?
- how to not let times where there are no activity take a lot of weight?
### enhancements
- create graph embedding of urls
- for each bin, calculate the metric per url
- instead of only looking at the specific website, take into account websites with similar usages,
  for example looking at same domain_cls usage in gaussian around bin, or looking at domain embeddings and looking at the activity in similar embeddings weighted by the distance in the embedding space

### NOTICE:
the data itself will use all domains, even ones that the person never used. this could be an issue. 
first of all the fact that the person doesnt use them is an indication. we 
- we might want to take the niche websites and sum them up
- we might want to remove them

IDEA!
- use different features for different people
- make an ensemble that can differentiate between different users
- take the people that get a wrong prediction and see if a classifier that is more "fringe" can classify them better
- for example another tree classifier that takes a smaller amount of features to give more opportunity to fringe websites
can create a classifier for each user type 
can take number of usages for each domain, and cluster people or PCA
clustering is good - I can create a classifier for each cluster, from each cluster take all of the available data for all of the visited domains, and create a classifier for them. use only data from those users or all users that used one of the websites, plus the general model, for each cluster - use cluster model and general model.
also - I can multiply the features by the log of usages

another idea is to simply average the most prominent websites weighted by the specific user usages, and the general usage

USER CLUSTER AS FEATURE - or PCA coefficients

an idea - see how chaotic are the subject's patterns, if it's too predictable then it might be a bot


## Options
### low activity enhancement:
- use selected features as initial centroids for domain clustering (spherical kmeans with initial centroids)
- impute selected features only for these people
### using highly visited websites:
- filter urls that more than 3-5 people visited
- simple option: give a score per website which is baysian probability
- deeper option : take all urls of highly visited websites
    - take 12-24 hour bits - and connect a trail from user to timebin to 1/0, then to all urls, where it increments the edge score by 1
    - get embedding (32-64)
    - use umap to turn to manifold with 8-16 dims
    - cluster the urls using kmeans or spectral clustering
    - for each person, give score for each cluster, how many urls from each cluster.


### Environment Initialization

In [ ]:
import sys
sys.path.append("/home/tom.touati/mafat-challenge/code/submission")
import importlib
import os
import ray
from modin.config import NPartitions,RangePartitioning
%matplotlib widget
os.environ["NEPTUNE_API_TOKEN"] = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiMGMyZjIyZC0xMjQzLTQxNjQtYjZjZC0wMTRiZmJmZmRlZjYifQ=="
    # !export MODIN_CPUS=2
# n_cpus = 8
plasma_store_size = 170*(1024**3)
heap_memory = 200*(1024**3)
# os.environ["MODIN_CPUS"] = str(n_cpus)
# os.environ["MODIN_ENGINE"] = "ray"
os.environ["MODIN_NPARTITIONS"] = "30"
os.environ["MODIN_RANGE_PARTITIONING"] = "True"
# os.environ["MODIN_MEMORY"] = str(plasma_store_size)
ray.init(num_cpus =30,ignore_reinit_error=True, object_store_memory=plasma_store_size,_memory=heap_memory)
# print(ray.cluster_resources())
import modin.pandas as mpd
from modin import config as cfg
print(vars(cfg))
NEPTUNE_MODE="sync"
params={}
%load_ext autoreload
%autoreload 2

### Data Loading & Preparation

In [12]:
%%writefile submission/load_and_prepare_input.py
import os
import sqlite3
# %matplotlib widget
import matplotlib.pyplot as plt
from multiprocessing import freeze_support
from modin.db_conn import ModinDatabaseConnection
import modin.pandas as mpd


# Modin will use Ray
# ray.init()
# NPartitions.put(16)
def load_domain_data_from_db(con, domain_cls=False, only_domain=False):
    try:
        device_ids_query = f"""SELECT Datetime,Device_ID,Domain_Name,Target from data
        WHERE Domain_Name != 1732927
        """

        # WHERE Domain_Name != 1732927 """
        df = mpd.read_sql(device_ids_query, con)._repartition()
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        raise


def load_cls_data_from_db(con, domain_cls=False, only_domain=False):
    try:
        device_ids_query = f"""SELECT Datetime,Device_ID,Domain_cls1,Domain_cls2,Domain_cls3,Domain_cls4,Target
        from data
        WHERE Domain_Name != 1732927
        """
        df = mpd.read_sql(device_ids_query, con)._repartition()
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        raise


def load_and_prepare_data(data_type="domain"):
    freeze_support()
    dbfile = '../../data/training_set.db'

    conn = ModinDatabaseConnection('sqlalchemy', f'sqlite:///{dbfile}')

    # Can use get_connection to get underlying sqlalchemy engine
    conn.get_connection()
    if data_type == "domain":
        db_df = load_domain_data_from_db(conn)
    elif data_type == "cls":
        db_df = load_cls_data_from_db(conn)

    print(db_df.head())
    del conn
    db_df['Datetime'] = mpd.to_datetime(db_df['Datetime'])
    db_df.set_index('Datetime', inplace=True)
    if data_type == "domain":
        db_df = db_df.astype({
            'Domain_Name': 'uint32',
            'Device_ID': 'uint32',
            'Target': 'uint8'
        })
    elif data_type == "cls":
        db_df = db_df.astype({
            'Device_ID': 'uint32',
            'Domain_cls1': 'uint32',
            'Domain_cls2': 'uint32',
            'Domain_cls3': 'uint32',
            'Domain_cls4': 'uint32',
            'Target': 'uint32'
        })
    return db_df


# prepare training data
import matplotlib.pyplot as plt
#train test split
from sklearn.model_selection import train_test_split


def get_train_test_devices(device_target_df, test_size=0.2, random_state=43):
    # Perform stratified split on device IDs
    train_device_ids, test_device_ids = train_test_split(
        device_target_df['Device_ID'],
        test_size=test_size,
        random_state=random_state,
        stratify=device_target_df['Target'])
    return train_device_ids, test_device_ids


def get_initial_train_data(db_df,
                           test_size=0.2,
                           random_state=42,
                           min_domain_devices=10,
                           n_devices_hist=False):
    device_targets = db_df.groupby("Device_ID")["Target"].first().reset_index()
    train_devices, test_device_ids = get_train_test_devices(
        device_targets, test_size=test_size, random_state=random_state)
    train_df = db_df[db_df["Device_ID"].isin(train_devices)]
    devices_per_domain = train_df.groupby("Domain_Name")["Device_ID"].nunique()

    domain_mask = devices_per_domain > min_domain_devices
    print(
        f"Percentage of domains with more than {min_domain_devices} devices: {domain_mask.mean()*100:.2f}%"
    )
    devices_per_domain = devices_per_domain[domain_mask]
    if n_devices_hist:
        hist = devices_per_domain.hist()
        # run["plots/domain_devices_hist"].upload(neptune.types.File.as_image(hist.figure))
        plt.show()
    train_df = train_df[train_df["Domain_Name"].isin(devices_per_domain.index)]
    return train_df, train_devices, test_device_ids, device_targets, devices_per_domain


# Add this line

Overwriting submission/load_and_prepare_input.py


In [ ]:
from submission.domain_timeseries_processing import *
from submission.utils import *
from submission.load_and_prepare_input import *
from submission.prepare_and_train_model import *
from submission.content_based_features import *
from submission.frequency_base_feats import *
from submission.cls_features import *

url_df = load_and_prepare_data()

params.update({
    "training_data": {
        "min_domain_devices": 10,
        "n_devices_hist": False,
        "test_size": 0.2,
        "random_state": 42
    }
})
train_df, train_devices, test_device_ids, device_targets, devices_per_valid_domain = get_initial_train_data(
    url_df, **params["training_data"])
train_df = train_df._repartition()

### Utils

In [13]:
%%writefile submission/utils.py
import numpy as np
import modin.pandas as mpd
from sklearn.preprocessing import MinMaxScaler, StandardScaler


def z_normalize_by_all(df,
                       train_devices,
                       per_column=True,
                       fillval=0,
                       fill_na_pre_transform=False,
                       scaler=None):
    if scaler is not None:
        if fill_na_pre_transform:
            df.fillna(fillval, inplace=True)
        df.iloc[:, :] = scaler.transform(
            df if per_column else df.values.reshape(-1, 1)).reshape(df.shape)
        if fillval:
            df.fillna(fillval, inplace=True)
        return
    scaler = StandardScaler()
    train_data = df.loc[train_devices]
    scaler.fit(train_data if per_column else train_data.values.reshape(-1, 1))

    # Transform all data using fitted scaler
    if fill_na_pre_transform:
        df.fillna(fillval, inplace=True)
    df.iloc[:, :] = scaler.transform(
        df if per_column else df.values.reshape(-1, 1)).reshape(df.shape)
    if fillval is not None:
        df.fillna(fillval, inplace=True)
    params = {
        "mean_":
        [float(x)
         for x in scaler.mean_],  # Convert to list for JSON serialization
        "var_": [float(x) for x in scaler.var_],
        "scale_": [float(x) for x in scaler.scale_],
        "n_samples_seen_": [int(x) for x in scaler.n_samples_seen_]
        if isinstance(scaler.n_samples_seen_, np.ndarray) else int(
            scaler.n_samples_seen_),
    }
    return params


def min_max_scale_all_values(df, train_devices, per_column=False, scaler=None):
    if scaler is not None:
        df.iloc[:, :] = scaler.transform(
            df if per_column else df.values.reshape(-1, 1)).reshape(df.shape)
        return
    # Create MinMaxScaler and fit on training data
    scaler = MinMaxScaler(feature_range=(-1, 1))
    train_data = df.loc[train_devices]
    scaler.fit(train_data if per_column else train_data.values.reshape(-1, 1))

    # Transform all data using fitted scaler
    df.iloc[:, :] = scaler.transform(
        df if per_column else df.values.reshape(-1, 1)).reshape(df.shape)

    # Save scaler parameters as JSON
    scaler_params = {
        "min_": float(scaler.min_[0]),  # Convert to native Python float
        "scale_": float(scaler.scale_[0]),
        "data_min_": float(scaler.data_min_[0]),
        "data_max_": float(scaler.data_max_[0]),
        "data_range_": float(scaler.data_range_[0]),
        # Convert tuple to list for JSON
        "feature_range": list(scaler.feature_range)
    }
    return scaler_params


from sklearn.impute import KNNImputer  # , IterativeImputer
from sklearn.linear_model import BayesianRidge
from scipy.sparse import csr_matrix


# from implicit.cpu.als import AlternatingLeastSquares
def impute_missing_values(final_scores_pivot, train_device_ids):
    # Use SoftImpute to fill missing values
    imputer = KNNImputer(n_neighbors=10)
    imputer.fit(final_scores_pivot.loc[train_device_ids])
    imputed_scores = imputer.transform(final_scores_pivot)
    # imputer = AlternatingLeastSquares(factors=10, regularization=0.01, iterations=10,random_state=0)
    # # imputer = MissForest(max_depth=6,max_features=0.8, random_state=0)
    # imputer.fit( user_items = csr_matrix(final_scores_pivot.loc[train_device_ids].values))

    # imputed_scores = imputer.recommend_all(csr_matrix(final_scores_pivot.values))
    return imputed_scores


import gc
import ctypes
import sys


def cleanup_memory():
    """
    Force cleanup of memory by:
    1. Running garbage collection
    2. Attempting to release memory back to OS
    """
    # Force garbage collection
    gc.collect()

    # Attempt to release memory back to the OS
    if sys.platform.startswith('linux'):
        libc = ctypes.CDLL('libc.so.6')
        # MALLOC_TRIM(0) releases memory back to OS if possible
        print(libc.malloc_trim(0))

Overwriting submission/utils.py


## Clustering

In [2]:
#load best features of cls and domain
#load usage proportions for domains
#use hdbscan with these initial centroids
# get score for each cluster by weighing the usage proportions


9